# Data Cleaning

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import re
pd.set_option('display.max_columns', None)

In [2]:
#Load housing data
acs_housing = pd.read_csv("ACS_HOUSING.csv", header= 1)

In [4]:
acs_housing.head()

,Geography,Geographic Area Name,Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage,Margin of Error!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage,"Estimate!!Less than \$50,000","Margin of Error!!Less than \$50,000","Estimate!!\$50,000 to \$99,999","Margin of Error!!\$50,000 to \$99,999","Estimate!!\$100,000 to \$299,999","Margin of Error!!\$100,000 to \$299,999","Estimate!!\$300,000 to \$499,999","Margin of Error!!\$300,000 to \$499,999","Estimate!!\$500,000 to \$749,999","Margin of Error!!\$500,000 to \$749,999","Estimate!!\$750,000 to \$999,999","Margin of Error!!\$750,000 to \$999,999","Estimate!!\$1,000,000 or more","Margin of Error!!\$1,000,000 or more",Estimate!!Median (dollars),Margin of Error!!Median (dollars),Estimate!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan,Margin of Error!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan,Estimate!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan!!Only Second mortgage,Margin of Error!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan!!Only Second mortgage,Estimate!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan!!Only Home equity loan,Margin of Error!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan!!Only Home equity loan,Estimate!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan!!Both second mortgage and home equity loan,Margin of Error!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan!!Both second mortgage and home equity loan,Estimate!!MORTGAGE STATUS!!No second mortgage and no home equity loan,Margin of Error!!MORTGAGE STATUS!!No second mortgage and no home equity loan,Estimate!!MORTGAGE STATUS!!Home equity loan without a primary mortgage,Margin of Error!!MORTGAGE STATUS!!Home equity loan without a primary mortgage,"Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Less than \$10,000","Margin of Error!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Less than \$10,000","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$10,000 to \$24,999","Margin of Error!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$10,000 to \$24,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$25,000 to \$34,999","Margin of Error!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$25,000 to \$34,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$35,000 to \$49,999","Margin of Error!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$35,000 to \$49,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$50,000 to \$74,999","Margin of Error!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$50,000 to \$74,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$75,000 to \$99,999","Margin of Error!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$75,000 to \$99,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$100,000 to \$149,999","Margin of Error!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$100,000 to \$149,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$150,000 or more","Margin of Error!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$150,000 or more",Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars),Margin of 

In [5]:
#Load rental data 
acs_rent = pd.read_csv("ACS_RENT copy.csv")

In [6]:
#Function to extract the location type of the municipality
def extract_city(location):
    loc = re.split(' CDP,| city,| town,| municipality,| village,| borough,| township,', location)
    return loc[0]

In [7]:
#merge rental data and housing data 
acs_full = pd.merge(acs_housing, acs_rent, how='inner', on='Geography')

#drop columns that calculate margin of error 
cols_to_drop = acs_full.filter(like='Margin of Error!!').columns
acs_full = acs_full.drop(columns=cols_to_drop)

#take only city name without the city type
acs_full['City Name'] = acs_full['Geographic Area Name_x'].apply(extract_city)


In [8]:
#extract state name from Geographic Area Feature
acs_full['State'] = [acs_full["Geographic Area Name_x"][i].split(", ")[1] for i in np.arange(len(acs_full))]

In [9]:
#Extract out census geocode 
acs_full['new_geocode'] = acs_full['Geography'].str.split('US')
acs_full["GEOID"] = [int(acs_full['new_geocode'][i][1]) for i in np.arange(len(acs_full))]

In [11]:
#import population and density data 
pop_data = pd.read_csv("uscities.csv")
pop_data.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18908608,11080.3
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,11922389,3184.7
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8497759,4614.5
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,6080145,4758.9
4,Houston,Houston,TX,Texas,48201,Harris,29.7860,-95.3885,5970127,1384.0


In [12]:
#merge population and density data onto housing and rental price data 
acs_city = pd.merge(acs_full, pop_data, how='left', left_on=['City Name', 'State'], right_on=['city_ascii', 'state_name'])
acs_city.head()

,Geography,Geographic Area Name_x,Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage,"Estimate!!Less than \$50,000","Estimate!!\$50,000 to \$99,999","Estimate!!\$100,000 to \$299,999","Estimate!!\$300,000 to \$499,999","Estimate!!\$500,000 to \$749,999","Estimate!!\$750,000 to \$999,999","Estimate!!\$1,000,000 or more",Estimate!!Median (dollars),Estimate!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan,Estimate!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan!!Only Second mortgage,Estimate!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan!!Only Home equity loan,Estimate!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan!!Both second mortgage and home equity loan,Estimate!!MORTGAGE STATUS!!No second mortgage and no home equity loan,Estimate!!MORTGAGE STATUS!!Home equity loan without a primary mortgage,"Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Less than \$10,000","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$10,000 to \$24,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$25,000 to \$34,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$35,000 to \$49,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$50,000 to \$74,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$75,000 to \$99,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$100,000 to \$149,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$150,000 or more",Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars),Estimate!!RATIO OF VALUE TO HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Less than 2.0,Estimate!!RATIO OF VALUE TO HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!2.0 to 2.9,Estimate!!RATIO OF VALUE TO HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!3.0 to 3.9,Estimate!!RATIO OF VALUE TO HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!4.0 or more,Estimate!!RATIO OF VALUE TO HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Not computed,Estimate!!MONTHLY HOUSING COSTS!!Less than \$200,Estimate!!MONTHLY HOUSING COSTS!!\$200 to \$399,Estimate!!MONTHLY HOUSING COSTS!!\$400 to \$599,Estimate!!MONTHLY HOUSING COSTS!!\$600 to \$799,Estimate!!MONTHLY HOUSING COSTS!!\$800 to \$999,"Estimate!!MONTHLY HOUSING COSTS!!\$1,000 to \$1,499","Estimate!!MONTHLY HOUSING COSTS!!\$1,500 to \$1,999","Estimate!!MONTHLY HOUSING COSTS!!\$2,000 to \$2,499","Estimate!!MONTHLY HOUSING COSTS!!\$2,500 to \$2,999","Estimate!!MONTHLY HOUSING COSTS!!\$3,000 or more",Estimate!!MONTHLY HOUSING COSTS!!Median (dollars),"Estimate!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Less than \$20,000","Estimate!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Less than \$20,000!!Less than 20 percent","Estimate!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Less than \$20,000!!20 to 29 percent","Estimate!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Less than \$20,000!!30 percent or more","Estimate!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!\$20,000 to \$34,999","Estimate!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!\$20,000 to \$34,999!!Less than 20 percent","Estimate!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!\$20,000 to \$34,999!!20 to 29 percent","Estimate!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!\$20,000 to \$34,999!!30 percent or more","Estimate!!MONTHLY HOUSI

In [22]:
#Load zoning data 
nzlud= pd.read_csv("nzlud_muni.csv").iloc[:, :-3]

In [23]:
#merge zoning data onto housing, rental, and population data 
result = pd.merge(nzlud, acs_city, how='left', on='GEOID')

In [24]:
#keep only data that matches between the zoning data and the housing, rental, and population data 
result_final = result[result['new_geocode'].notna()]

In [25]:
result_final

,GEOID,statename,place,type,restrict_sf_permit,restrict_mf_permit,limit_sf_units,limit_mf_units,limit_mf_dwellings,limit_mf_dwelling_units,min_lot_size,open_space,inclusionary,half_acre_less,half_acre_more,one_acre_more,two_acre_more,max_den_cat1,max_den_cat2,max_den_cat3,max_den_cat4,max_den_cat5,council_nz,planning_nz,countybrd_nz,pubhlth_nz,site_plan_nz,env_rev_nz,council_rz,planning_rz,zoning_rz,countybrd_rz,countyzone_rz,townmeet_rz,env_rev_rz,total_nz,total_rz,adu,height_ft_median,height_ft_mode,height_st_median,height_st_mode,parking_median,parking_mode,mf_per,maxden5,maxden4,maxden3,maxden2,maxden1,sindex1,sindex2,sindex3,sindex4,sindex5,sindex6,sindex7,sindex8,height_st_median_sc,height_st_mode_sc,sindex9,sindex10,sindex11,sindex9_st,sindex10_st,zri,zri_st,add_index,zri_up,zri_up_st,wt_all_2019,st_wt_all_2019,wt_msa_2019,st_wt_msa_2019,Geography,Geographic Area Name_x,Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage,"Estimate!!Less than \$50,000","Estimate!!\$50,000 to \$99,999","Estimate!!\$100,000 to \$299,999","Estimate!!\$300,000 to \$499,999","Estimate!!\$500,000 to \$749,999","Estimate!!\$750,000 to \$999,999","Estimate!!\$1,000,000 or more",Estimate!!Median (dollars),Estimate!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan,Estimate!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan!!Only Second mortgage,Estimate!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan!!Only Home equity loan,Estimate!!MORTGAGE STATUS!!With a mortgage and either a second mortgage or home equity loan!!Both second mortgage and home equity loan,Estimate!!MORTGAGE STATUS!!No second mortgage and no home equity loan,Estimate!!MORTGAGE STATUS!!Home equity loan without a primary mortgage,"Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Less than \$10,000","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$10,000 to \$24,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$25,000 to \$34,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$35,000 to \$49,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$50,000 to \$74,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$75,000 to \$99,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$100,000 to \$149,999","Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!\$150,000 or more",Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars),Estimate!!RATIO OF VALUE TO HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Less than 2.0,Estimate!!RATIO OF VALUE TO HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!2.0 to 2.9,Estimate!!RATIO OF VALUE TO HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!3.0 to 3.9,Estimate!!RATIO OF VALUE TO HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!4.0 or more,Estimate!!RATIO OF VALUE TO HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Not computed,Estimate!!MONTHLY HOUSING COSTS!!Less than \$200,Estimate!!MONTHLY HOUSING COSTS!!\$200 to \$399,Estimate!!MONTHLY HOUSING COSTS!!\$400 to \$599,Estimate!!MONTHLY HOUSING COSTS!!\$600 to \$799,Estimate!!MONTHLY HOUSING COSTS!!\$800 to \$999,"Estimate!!MONTHLY HOUSING COSTS!!\$1,000 to \$1,499","Estimate!!MONTHLY HOUSING COSTS!!\$1,500 to \$1,999","Estimate!!MONTHLY HOUSING COSTS!!\$2,000 to \$2,499","Estimate!!MONTHLY HOUSING COSTS!!\$2,500 to \$2,999","Estimate!!MONTHLY HOUSING COSTS!!\$3,000 or more",Estimate!!MONTHLY HOUSING COSTS!!Median (dollars),"Estimate!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Less than \$20,000","Estimate!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST

In [ ]:
results_final.to_csv('joined_data.csv', index  = False)